In [1]:
import pubchempy as pcp
import json

with open('./primekg_id_mapping.json', 'r') as f:
    id2name_drug = json.load(f)['id2name_drug']


In [ ]:
import time
from tqdm import tqdm

drugbank2pubchem = {}

for key in tqdm(id2name_drug.keys()):
    DBid = key
    drug_name = id2name_drug[key]
    while True:
        try:
            PCid = pcp.get_compounds(drug_name, 'name')
            drugbank2pubchem[DBid] = PCid
            break
        except pcp.PubChemHTTPError as e:
            if 'ServerBusy' in str(e):
                print('Server is busy. Retrying in 10 seconds...')
                time.sleep(10)
            else:
                raise e


In [14]:
# Search for the compound name
p = pcp.get_properties('IsomericSMILES', 1, 'cid')


In [16]:
p[0]

{'CID': 1, 'IsomericSMILES': 'CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C'}

In [26]:
cnt = 0
prime_kg_cid_set = set()

for key in tqdm(id2name_drug.keys()):
    try:
        prime_kg_cid_set.add(drugbank2pubchem[key][0].cid)
    except:
        cnt+=1
        continue

100%|██████████| 7957/7957 [00:00<00:00, 397490.26it/s]


In [42]:
drugbank2pubchem_cid = {key: drugbank2pubchem[key][0].cid for key in drugbank2pubchem.keys() if len(drugbank2pubchem[key]) > 0}

with open("prime_kg_db_cid.json", 'w') as f:
    json.dump(drugbank2pubchem_cid, f, indent=6)

In [28]:
len(prime_kg_cid_set)

7083

In [30]:
import glob
import pandas as pd

downstream_smiles_set = set()

# Path to the directory containing the CSV files
csv_dir = './finetune'

# Use glob to get a list of all CSV files in the directory
csv_files = glob.glob(csv_dir + '/*.csv')

# Loop over each file and extract the SMILES strings from the first column
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    smiles_col = df.iloc[:, 0]  # assuming the first column is the SMILES column
    downstream_smiles_set.update(smiles_col)



In [31]:
len(downstream_smiles_set)

53263

In [47]:
import time
from tqdm import tqdm

smile_cid = {}

for smile in tqdm(downstream_smiles_set):
    while True:
        try:
            PCid = pcp.get_compounds(smile, 'smiles')[0].cid
            smile_cid[smile] = PCid
            break
        except pcp.PubChemHTTPError as e:
            if 'ServerBusy' in str(e):
                print('Server is busy. Retrying in 1 seconds...')
                time.sleep(1)
            else:
                smile_cid[smile] = 'null'
                break


100%|██████████| 46598/46598 [7:31:09<00:00,  1.72it/s]   


In [48]:
import json

with open('./smile2cid_1.json', 'w') as f:
    json.dump(smile_cid, f, indent=6)

In [49]:
import json

with open('./smile2cid_1.json', 'r') as f:
    cid_1 = json.load(f).values()

In [50]:
import requests
from collections import defaultdict
from operator import itemgetter
from tqdm import tqdm
import re

# List of CIDs to query
cid_list = list(cid_1)

# Define the API endpoints for each entity type
endpoints = ['chemical_to_disease', 'chemical_to_gene', 'chemical_to_chemical']

# Initialize dictionaries to store the top 5 entities for each CID
top_compounds = defaultdict(list)
top_diseases = defaultdict(list)
top_genes = defaultdict(list)

# Loop over each CID in the list and query the top 5 compounds, diseases, and genes
for cid in tqdm(cid_list):
    if cid == 'null' or cid == None:
        continue
    for endpoint in endpoints:
        # Construct the API endpoint URL
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/rdf/query?graph=cooccurrence&entity1=CID{cid}&type={endpoint}"
        
        # Query the API and parse the response
        response = requests.get(url)
        lines = response.text
        pattern = r'<td><a href="(.*?)">(.*?)</a></td>\n\s*<td>(\d+)</td>'
        edges = re.findall(pattern, lines)
        
        # Sort the edges by score and keep the top 5
        try:
            sorted_edges = edges[:5]
        except:
            continue
        
        # Normalize the edge weights by score and store the results in the corresponding dictionary
        for edge in sorted_edges:
            entity_id = edge[0].split('/')[-1]
            score = int(edge[2])
            normalized_score = score / int(sorted_edges[0][2])  # normalize by the maximum score
            if endpoint == 'chemical_to_chemical':
                top_compounds[cid].append((entity_id, normalized_score))
            elif endpoint == 'chemical_to_disease':
                top_diseases[cid].append((entity_id, normalized_score))
            elif endpoint == 'chemical_to_gene':
                top_genes[cid].append((entity_id, normalized_score))



100%|██████████| 46598/46598 [4:50:04<00:00,  2.68it/s]   


In [51]:
with open('./chemical_chemical_2.json', 'w') as f:
    json.dump(top_compounds, f, indent=6)

with open('./chemical_disease_2.json', 'w') as f:
    json.dump(top_diseases, f, indent=6)

with open('./chemical_gene_2.json', 'w') as f:
    json.dump(top_genes, f, indent=6)